<a href="https://colab.research.google.com/github/kavish-24/Konkani_Mentall_Health/blob/main/AudioHandling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
pip install openai-whisper pydub

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 803.2/803.2 kB 17.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for openai-whisper: filename=openai_whisper-20250625-py3-none-any.whl size=803979 sha256=0d1d7c2b00514abd1176042947dbcb2327a663c71ebfd0ece18646e43e6c44f1
  Stored in directory: /root/.cache/pip/wheels/61/d2/20/09ec9bef734d126cba375b15898010b6cc28578d8afdde5869
Successfully built openai-whisper


In [ ]:
pip install openai-whisper

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 803.2/803.2 kB 21.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for openai-whisper: filename=openai_whisper-20250625-py3-none-any.whl size=803979 sha256=b762e624668be70d6834a57a55083c2a7874c72c01aa845f2d42daa25ca6be90
  Stored in directory: /root/.cache/pip/wheels/61/d2/20/09ec9bef734d126cba375b15898010b6cc28578d8afdde5869
Successfully built openai-whisper


In [ ]:
import whisper
from pydub import AudioSegment
import os

def split_audio_on_word_boundaries(audio_path, max_duration=30, output_dir="segments"):
    """
    Split audio into segments of max_duration seconds, ending at word boundaries.

    Args:
        audio_path: Path to input audio file
        max_duration: Maximum segment duration in seconds (default: 30)
        output_dir: Directory to save output segments
    """

    # Create output directory if it doesn't exist
    os.makedirs(output_dir, exist_ok=True)

    # Load Whisper model (use 'base' for speed, 'small'/'medium' for accuracy)
    print("Loading Whisper model...")
    model = whisper.load_model("base")

    # Transcribe audio with word-level timestamps
    print(f"Transcribing {audio_path}...")
    result = model.transcribe(audio_path, word_timestamps=True)

    # Load audio file
    print("Loading audio file...")
    audio = AudioSegment.from_file(audio_path)

    # Extract all words with timestamps
    words = []
    for segment in result['segments']:
        if 'words' in segment:
            words.extend(segment['words'])

    if not words:
        print("No words detected in audio!")
        return

    # Split into segments
    segments = []
    current_start = 0
    max_duration_ms = max_duration * 1000

    for i, word in enumerate(words):
        word_end_ms = word['end'] * 1000

        # Check if adding this word would exceed max duration
        if word_end_ms - (current_start * 1000) >= max_duration_ms:
            # End segment at previous word boundary
            if i > 0:
                segment_end = words[i-1]['end']
                segments.append((current_start, segment_end))
                current_start = segment_end
            else:
                # First word itself exceeds duration, include it anyway
                segment_end = word['end']
                segments.append((current_start, segment_end))
                current_start = segment_end

    # Add final segment
    if current_start < words[-1]['end']:
        segments.append((current_start, words[-1]['end']))

    # Export segments
    print(f"\nCreating {len(segments)} segments...")
    base_name = os.path.splitext(os.path.basename(audio_path))[0]

    for idx, (start, end) in enumerate(segments, 1):
        start_ms = int(start * 1000)
        end_ms = int(end * 1000)

        segment_audio = audio[start_ms:end_ms]
        output_path = os.path.join(output_dir, f"{base_name}_segment_{idx:03d}.mp3")

        segment_audio.export(output_path, format="mp3")
        duration = (end - start)
        print(f"Segment {idx}: {duration:.2f}s ({start:.2f}s - {end:.2f}s) -> {output_path}")

    print(f"\nDone! Created {len(segments)} segments in '{output_dir}' directory")
    return segments


def split_audio_aggressive(audio_path, max_duration=30, output_dir="segments"):
    """
    Alternative approach: More aggressive splitting for very fast speech.
    Uses silence detection as fallback if word boundaries are too far apart.
    """
    from pydub.silence import detect_nonsilent

    os.makedirs(output_dir, exist_ok=True)

    print("Loading Whisper model...")
    model = whisper.load_model("base")

    print(f"Transcribing {audio_path}...")
    result = model.transcribe(audio_path, word_timestamps=True)

    print("Loading audio file...")
    audio = AudioSegment.from_file(audio_path)

    # Extract words with timestamps
    words = []
    for segment in result['segments']:
        if 'words' in segment:
            words.extend(segment['words'])

    if not words:
        print("No words detected! Falling back to silence detection...")
        return split_on_silence_only(audio_path, max_duration, output_dir)

    # Detect silence periods (as backup split points)
    nonsilent_ranges = detect_nonsilent(audio, min_silence_len=100, silence_thresh=-40)

    segments = []
    current_start = 0
    max_duration_ms = max_duration * 1000

    for i, word in enumerate(words):
        word_end_ms = word['end'] * 1000
        current_duration = word_end_ms - (current_start * 1000)

        if current_duration >= max_duration_ms:
            # Try to find a word boundary close to max_duration
            target_time = current_start * 1000 + max_duration_ms

            # Find nearest word end before target time
            best_split = None
            for j in range(i, -1, -1):
                candidate_end = words[j]['end'] * 1000
                if candidate_end <= target_time:
                    best_split = words[j]['end']
                    break

            if best_split:
                segments.append((current_start, best_split))
                current_start = best_split
            else:
                # Use current word as split point
                segments.append((current_start, word['end']))
                current_start = word['end']

    # Add final segment
    if current_start < words[-1]['end']:
        segments.append((current_start, words[-1]['end']))

    # Export segments
    print(f"\nCreating {len(segments)} segments...")
    base_name = os.path.splitext(os.path.basename(audio_path))[0]

    for idx, (start, end) in enumerate(segments, 1):
        start_ms = int(start * 1000)
        end_ms = int(end * 1000)

        segment_audio = audio[start_ms:end_ms]
        output_path = os.path.join(output_dir, f"{base_name}_segment_{idx:03d}.mp3")

        segment_audio.export(output_path, format="mp3")
        duration = (end - start)
        print(f"Segment {idx}: {duration:.2f}s -> {output_path}")

    print(f"\nDone! Created {len(segments)} segments")
    return segments


# Example usage
if __name__ == "__main__":
    # ============= MANUAL PATH SETTINGS =============
    # Set your input audio file path here
    audio_file = r"/content/drive/MyDrive/Anju Project (1)/Audio Prudent media (1)/August 2017 (1)/Konkani Prime News_100817.wav"

    # Set your output directory path here
    output_directory = r"/content/drive/MyDrive/Anju Project (1)/Audio Prudent media (1)/audio segment/10 Aug"

    # Maximum duration per segment in seconds
    max_segment_duration = 30
    # ================================================

    print(f"Input audio: {audio_file}")
    print(f"Output directory: {output_directory}")
    print(f"Max segment duration: {max_segment_duration} seconds\n")

    # Basic splitting (recommended)
    segments = split_audio_on_word_boundaries(
        audio_file,
        max_duration=max_segment_duration,
        output_dir=output_directory
    )

    # Or use aggressive splitting for very fast speech
    # segments = split_audio_aggressive(
    #     audio_file,
    #     max_duration=max_segment_duration,
    #     output_dir=output_directory
    # )

/usr/local/lib/python3.12/dist-packages/pydub/utils.py:300: SyntaxWarning: invalid escape sequence '\('
  m = re.match('([su]([0-9]{1,2})p?) \(([0-9]{1,2}) bit\)$', token)
/usr/local/lib/python3.12/dist-packages/pydub/utils.py:301: SyntaxWarning: invalid escape sequence '\('
  m2 = re.match('([su]([0-9]{1,2})p?)( \(default\))?$', token)
/usr/local/lib/python3.12/dist-packages/pydub/utils.py:310: SyntaxWarning: invalid escape sequence '\('
  elif re.match('(flt)p?( \(default\))?$', token):
/usr/local/lib/python3.12/dist-packages/pydub/utils.py:314: SyntaxWarning: invalid escape sequence '\('
  elif re.match('(dbl)p?( \(default\))?$', token):


Input audio: /content/drive/MyDrive/Anju Project (1)/Audio Prudent media (1)/August 2017 (1)/Konkani Prime News_100817.wav
Output directory: /content/drive/MyDrive/Anju Project (1)/Audio Prudent media (1)/audio segment/10 Aug
Max segment duration: 30 seconds

Loading Whisper model...


100%|███████████████████████████████████████| 139M/139M [00:02<00:00, 49.0MiB/s]


Transcribing /content/drive/MyDrive/Anju Project (1)/Audio Prudent media (1)/August 2017 (1)/Konkani Prime News_100817.wav...
Loading audio file...

Creating 15 segments...
Segment 1: 29.64s (0.00s - 29.64s) -> /content/drive/MyDrive/Anju Project (1)/Audio Prudent media (1)/audio segment/10 Aug/Konkani Prime News_100817_segment_001.mp3
Segment 2: 29.40s (29.64s - 59.04s) -> /content/drive/MyDrive/Anju Project (1)/Audio Prudent media (1)/audio segment/10 Aug/Konkani Prime News_100817_segment_002.mp3
Segment 3: 29.78s (59.04s - 88.82s) -> /content/drive/MyDrive/Anju Project (1)/Audio Prudent media (1)/audio segment/10 Aug/Konkani Prime News_100817_segment_003.mp3
Segment 4: 29.68s (88.82s - 118.50s) -> /content/drive/MyDrive/Anju Project (1)/Audio Prudent media (1)/audio segment/10 Aug/Konkani Prime News_100817_segment_004.mp3
Segment 5: 29.38s (118.50s - 147.88s) -> /content/drive/MyDrive/Anju Project (1)/Audio Prudent media (1)/audio segment/10 Aug/Konkani Prime News_100817_segment_005

In [3]:
import os
import whisper
from pathlib import Path

def transcribe_mp3_directory(input_dir, output_dir=None, model_size="medium", language=None):
    """
    Transcribe all MP3 files in a directory using Whisper model.

    Args:
        input_dir: Path to directory containing MP3 files
        output_dir: Path to directory where transcripts will be saved (default: same as input_dir)
        model_size: Whisper model size (default: "medium")
        language: Language code (None for auto-detection, "mr" for Marathi, "hi" for Hindi)
    """

    # Load Whisper model
    print(f"Loading Whisper {model_size} model...")
    model = whisper.load_model(model_size)

    # Set up directories
    input_path = Path(input_dir)
    if output_dir is None:
        output_path = input_path
    else:
        output_path = Path(output_dir)
        output_path.mkdir(parents=True, exist_ok=True)

    # Get all MP3 files
    mp3_files = list(input_path.glob("*.mp3"))

    if not mp3_files:
        print(f"No MP3 files found in {input_dir}")
        return

    print(f"Found {len(mp3_files)} MP3 files to transcribe")
    print(f"Language setting: {'Auto-detect' if language is None else language}\n")

    # Transcribe each file
    successful = 0
    empty = 0

    for i, mp3_file in enumerate(mp3_files, 1):
        print(f"[{i}/{len(mp3_files)}] Transcribing: {mp3_file.name}")

        try:
            # Transcribe with additional options for better results
            transcribe_options = {
                "task": "transcribe",
                "fp16": False,  # Use FP32 for better accuracy
                "temperature": 0.0,  # Deterministic output
                "best_of": 5,  # Try multiple decodings
                "beam_size": 5,  # Better search
            }

            if language:
                transcribe_options["language"] = language

            result = model.transcribe(str(mp3_file), **transcribe_options)

            # Check if transcript is empty
            transcript_text = result["text"].strip()

            if not transcript_text:
                print(f"⚠ Empty transcript (possibly silent audio)")
                empty += 1
            else:
                successful += 1
                print(f"✓ Detected language: {result.get('language', 'unknown')}")
                print(f"  Text preview: {transcript_text[:100]}{'...' if len(transcript_text) > 100 else ''}")

            # Save transcript (even if empty, for tracking)
            transcript_file = output_path / f"{mp3_file.stem}_transcript.txt"
            with open(transcript_file, "w", encoding="utf-8") as f:
                f.write(transcript_text)
                f.write(f"\n\n[Detected Language: {result.get('language', 'unknown')}]")

            print(f"  Saved to: {transcript_file.name}\n")

        except Exception as e:
            print(f"✗ Error transcribing {mp3_file.name}: {str(e)}\n")
            continue

    print("="*60)
    print("Transcription complete!")
    print(f"Successful: {successful} | Empty: {empty} | Total: {len(mp3_files)}")
    print("="*60)

if __name__ == "__main__":
    # For Konkani audio, try these options:

    # OPTION 1: Auto-detect language (recommended for Konkani)
    # Whisper will try to detect the closest language
    input_directory = "/content/drive/MyDrive/Anju Project (1)/Audio Prudent media (1)/audio segment/10 Aug"
    output_directory = "/content/drive/MyDrive/Anju Project (1)/Audio Prudent media (1)/transcript segment/10 Aug"

    transcribe_mp3_directory(
    input_dir=input_directory,
    output_dir=output_directory,
    model_size="large-v2",
    language="mr"
)


    # OPTION 2: Try Hindi (closer to Konkani than Marathi)
    # Uncomment below to try:
    # transcribe_mp3_directory(
    #     input_dir=input_directory,
    #     output_dir=output_directory,
    #     model_size="medium",
    #     language="hi"  # Hindi
    # )

    # OPTION 3: Try Marathi (as originally requested)
    # Uncomment below to try:
    # transcribe_mp3_directory(
    #     input_dir=input_directory,
    #     output_dir=output_directory,
    #     model_size="medium",
    #     language="mr"  # Marathi
    # )

Loading Whisper large-v2 model...


100%|█████████████████████████████████████| 2.87G/2.87G [00:36<00:00, 84.4MiB/s]


Found 15 MP3 files to transcribe
Language setting: mr

[1/15] Transcribing: Konkani Prime News_100817_segment_001.mp3
✓ Detected language: mr
  Text preview: नमस्कार पोले प्रुडंड खोब्रोग टैंकर वालेंग न धर्बान गौय भर च वाटर टैंकर आचे ट्रांस्पोर्ट डिपाट्मेंट क...
  Saved to: Konkani Prime News_100817_segment_001_transcript.txt

[2/15] Transcribing: Konkani Prime News_100817_segment_002.mp3
✓ Detected language: mr
  Text preview: पुर्रीकार खैंश सगल्यां ब्रश्ट मुख्यमुख्यमंत्री पुर्रीकार खैंश सगल्यां ब्रश्ट मुख्यमुख्यमंत्री पुर्री...
  Saved to: Konkani Prime News_100817_segment_002_transcript.txt

[3/15] Transcribing: Konkani Prime News_100817_segment_003.mp3
✓ Detected language: mr
  Text preview: मिलिंदाची कार्ये गर्ध्यां ग्यों फातोडेर सांध बोरिये मिरें साथ वरांची वारी अनि भारतांट फुटसाला गेतले ...
  Saved to: Konkani Prime News_100817_segment_003_transcript.txt

[4/15] Transcribing: Konkani Prime News_100817_segment_004.mp3
✓ Detected language: mr
  Text preview: कलसा बंडूरा परकल्बाचे क